# DS 3000 Quiz 1

#! snip: Quiz1
# $\color{red}{\text{SOLUTIONS}}$ 
#! snip-end

Due by: Tuesday Oct 9 @ 11:59 PM EST

Time Limit: You have 2 hours to complete the assignment once started

## Instructions

This quiz has 100 points total.

- You are welcome to post a private note on piazza, but to keep a consistent testing environment for all students we are unlikely to provide assistance.
- You may not contact other students with information about this this quiz
    - even saying "it was easy/hard" in a general sense can introduce a bias in favor of students who take the quiz earlier or later
- Under no circumstances should you share a copy of this quiz with anyone who isn't a member of the course staff.
- Take this quiz with open notes and feel free to access any online resource / documentation you'd like.  

### Submission Instructions
After completing the quiz below, please follow the instructions below to submit:
1. "Kernel" -> "Restart & Run All"
1. save your quiz file to this latest version
1. upload the `.ipynb` to gradescope **before** clicking submit
1. ensure that you can see your jupyter notebook in the gradescope interface after clicking "submit"

We specify the last note above as gradescope has allowed students to "submit" without uploading a file.  It is your responsibility to ensure that you've actually submitted a file.

In [1]:
#! snip: Quiz1

!python3 -m snip_copy Quiz1_Solutions.ipynb

#! snip-end

snip-copied: Quiz1.ipynb


### Academic Integrity Pledge

Input your name below to sign the Academic Integrity Pledge before continuing with the quiz. Failure to do so will result in a score of **0**.

In [2]:
name = 'Student Name Here'
print(f'I, {name}, declare that the following work is entirely my own, and that I did not copy or seek help from any students who have currently or previously taken this course, nor from any online source other than private messages between myself and the professor on Piazza/via email.')

I, Student Name Here, declare that the following work is entirely my own, and that I did not copy or seek help from any students who have currently or previously taken this course, nor from any online source other than private messages between myself and the professor on Piazza/via email.


In [3]:
# the following modules will be necessary to complete the quiz
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

# Part 1: Dictionary API (50 points)

Using [this dictionary API](https://dictionaryapi.dev/) create the following dataframe by searching for the words `hello`, `data` and `science`.

Note that your searches may return multiple words, multiple definitions or multiple pronounciations.  Where necessary, always select the first.  


|   |    word |                                                          url_pronounce |                                                                                                                                                                                              definition |
|--:|--------:|-----------------------------------------------------------------------:|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| 0 |   hello |     https://api.dictionaryapi.dev/media/pronunciations/en/hello-au.mp3 |                                                                                                                                                                     "Hello!" or an equivalent greeting. |
| 1 |    data |   https://api.dictionaryapi.dev/media/pronunciations/en/data-au-nz.mp3 | (plural: data) A measurement of something on a scale understood by both the recorder (a person or device) and the reader (another person or device). The scale is arbitrarily defined, such as from ... |
| 2 | science | https://api.dictionaryapi.dev/media/pronunciations/en/science-1-ca.mp3 |                                                A particular discipline or branch of learning, especially one dealing with measurable or systematic principles rather than intuition or natural ability. |

**Note:** Because each row of the pandas dataframe contains so many characters, you may find that:

    pd.options.display.max_colwidth = 200
    
allows you to see the whole thing.

**Note Also:** Your response need not build any functions, but be sure to name variables appropriately and document your process.

In [4]:
#! snip: Quiz1

# Create a word list of the words we want to search for
word_list = ['hello', 'data', 'science']

# Initialize our dataframe
word_df = pd.DataFrame(columns=['word', 'url_pronounce', 'definition'])

# Iterate over the word list
for word in word_list:
    # Source: https://dictionaryapi.dev/
    url = f'https://api.dictionaryapi.dev/api/v2/entries/en/{word}'
    word_data = requests.get(url).text
    word_dict = json.loads(word_data)
    
    # Initialize a dictionary to store data on each word at intermediate points
    helper_dict = dict()
    
    # Use dictionary indexing to find desired values
    # There are 0's in between key names since some key values are lists
    helper_dict['word'] = word
    helper_dict['url_pronounce'] = word_dict[0]['phonetics'][0]['audio']
    helper_dict['definition'] = word_dict[0]['meanings'][0]['definitions'][0]['definition']
    
    word_df = pd.concat([word_df, pd.Series(helper_dict).to_frame().T], ignore_index = True)

pd.options.display.max_colwidth = 200
word_df

#! snip-end

,word,url_pronounce,definition
0,hello,https://api.dictionaryapi.dev/media/pronunciations/en/hello-au.mp3,"""Hello!"" or an equivalent greeting."
1,data,https://api.dictionaryapi.dev/media/pronunciations/en/data-au-nz.mp3,"(plural: data) A measurement of something on a scale understood by both the recorder (a person or device) and the reader (another person or device). The scale is arbitrarily defined, such as from ..."
2,science,https://api.dictionaryapi.dev/media/pronunciations/en/science-1-ca.mp3,"A particular discipline or branch of learning, especially one dealing with measurable or systematic principles rather than intuition or natural ability."


,word,url_pronounce,definition
0,hello,https://api.dictionaryapi.dev/media/pronunciations/en/hello-au.mp3,"""Hello!"" or an equivalent greeting."
1,data,https://api.dictionaryapi.dev/media/pronunciations/en/data-au-nz.mp3,"(plural: data) A measurement of something on a scale understood by both the recorder (a person or device) and the reader (another person or device). The scale is arbitrarily defined, such as from ..."
2,science,https://api.dictionaryapi.dev/media/pronunciations/en/science-1-ca.mp3,"A particular discipline or branch of learning, especially one dealing with measurable or systematic principles rather than intuition or natural ability."


# Part 2: Web Scraping Korean Dramas (50 points)

Your goal is to build a data frame that includes two columns: `category` and `movie` based on the 50 best Korean Dramas according to [this website](https://www.marieclaire.com/culture/a26895105/best-korean-dramas/). To help you, the actual web scraping part of this problem is done in the first code cell below, along with the first step of cleaning the data. The result is a list of headers from the web site. Note:

- certain elements in the list are the categories
- all elements trailing a category belong to that category until a new category appears

**Note:** the below are directions for one way to accomplish the task. If you can think of a faster way to do it, please do so!

Create two empty lists, `kdramas_cats` and `kdramas_movs`. Then, loop through the `headers` and build out the lists such that the `kdramas_cats` contains the category corresponding to each movie and `kdramas_movs` contains all the movies. Then, use these lists to create a data frame with a column called `category` (where the `kdramas_cats` data are stored) and a column called `movie` (where the `kdramas_movs` data are stored). Make sure you clean the data so that:

- the categories do not have the `' Korean Dramas'` part of their string
- the last element of `headers` is not included (since it is not a movie, but rather an advertisment that happened to share the `<h2>` tag)

When you are done, print the entire data frame to ensure it all worked.

**Note:** Your response need not build any functions, but be sure to name variables appropriately and document your process.

In [5]:
# the url, scraper, and soup object
url = 'https://www.marieclaire.com/culture/a26895105/best-korean-dramas/'
html_text = requests.get(url).text
soup = BeautifulSoup(html_text)

# parsing the data to get only the headers
headers = soup.find_all('h2')
headers

[<h2 class="article-body__section" id="section-action-thriller-korean-dramas"><span>Action/Thriller Korean Dramas</span></h2>,
 <h2 id="apos-squid-game-apos-2">'Squid Game'</h2>,
 <h2 id="apos-vincenzo-apos-2">'Vincenzo'</h2>,
 <h2 id="apos-happiness-apos-2">'Happiness'</h2>,
 <h2 id="apos-all-of-us-are-dead-apos-2">'All of Us Are Dead'</h2>,
 <h2 id="apos-my-name-apos-2">'My Name'</h2>,
 <h2 id="apos-d-p-apos-2">'D.P'</h2>,
 <h2 id="apos-weak-hero-class-1-apos-2">'Weak Hero Class 1'</h2>,
 <h2 id="apos-bloodhounds-apos-2">'Bloodhounds'</h2>,
 <h2 class="article-body__section" id="section-romance-korean-dramas"><span>Romance Korean Dramas</span></h2>,
 <h2 id="apos-crash-landing-on-you-apos-2">'Crash Landing on You'</h2>,
 <h2 id="apos-business-proposal-apos-2">'Business Proposal'</h2>,
 <h2 id="apos-hometown-cha-cha-cha-apos-2">'Hometown Cha-Cha-Cha'</h2>,
 <h2 id="apos-coffee-prince-apos-2">'Coffee Prince'</h2>,
 <h2 id="apos-boys-over-flowers-apos-2">'Boys Over Flowers'</h2>,
 <h2 i

In [6]:
#! snip: Quiz1

# create empty lists
kdrama_cats = []
kdrama_movs = []

# loop through headers
for item in headers:
    # check if header is a category
    if item.span is not None:
        # if so, set this to be the current category, drop the ' Korean Dramas'
        current_cat = item.text.replace(' Korean Dramas', '')
    else:
        # if not, we have started the movies within the current category
        # append both lists with current category and current movie
        kdrama_cats.append(current_cat)
        kdrama_movs.append(item.text)

# put the lists into a dictionary, ignoring the last item (which is not a movie)
kdrama_dict = {'category': kdrama_cats[:-1],
               'movie': kdrama_movs[:-1]}

# cast the dictionary to a data frame
kdrama_df = pd.DataFrame.from_dict(kdrama_dict)
kdrama_df

#! snip-end

,category,movie
0,Action/Thriller,'Squid Game'
1,Action/Thriller,'Vincenzo'
2,Action/Thriller,'Happiness'
3,Action/Thriller,'All of Us Are Dead'
4,Action/Thriller,'My Name'
5,Action/Thriller,'D.P'
6,Action/Thriller,'Weak Hero Class 1'
7,Action/Thriller,'Bloodhounds'
8,Romance,'Crash Landing on You'
9,Romance,'Business Proposal'
